# Assess the marginal performance of every feature

In [1]:
import math

import pandas
import sklearn.metrics
import scipy.stats
from statsmodels.sandbox.stats.multicomp import multipletests

In [2]:
feature_type_df = pandas.read_table('data/matrix/feature-type.tsv')
feature_df = pandas.read_table('data/matrix/features.tsv.bz2')
features = list(feature_type_df['feature'])

In [3]:
feature_df.head(2)

,hetnet,compound_id,disease_id,status,disease_name,compound_name,prior_prob,CbG,CcSE,CdG,...,CuGuDpCpD,CuGuDpCtD,CuGuDpSpD,CuGuDrD,CuGuDrDrD,CuGuDtCpD,CuGuDtCtD,CuGuDuGaD,CuGuDuGdD,CuGuDuGuD
0,rephetio-v2.0,DB00014,DOID:0050741,0,alcohol dependence,Goserelin,0.009801,2,249,0,...,0.0,0.0,0.000413,0.0,0.0,0.0,0.0,0.0,0.000212,0.001160
1,rephetio-v2.0_perm-5,DB00014,DOID:0050741,0,alcohol dependence,Goserelin,0.009801,2,249,0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001268,0.003163


In [4]:
def compute_metrics(y_true, y_score):
    series = pandas.Series()
    series['nonzero'] = (y_score > 0).mean()
    series['auroc'] = sklearn.metrics.roc_auc_score(y_true, y_score)
    series['auprc'] = sklearn.metrics.average_precision_score(y_true, y_score)
    return series

def columnar_performance(df):
    y_true = df['status']
    perf_df = df[features].apply(lambda x: compute_metrics(y_true, x), axis='index')
    perf_df = perf_df.T
    perf_df.index.name = 'feature'
    return perf_df.reset_index()

perf_df = feature_df.groupby('hetnet').apply(columnar_performance).reset_index(level='hetnet')
perf_df = perf_df.merge(feature_type_df)
perf_df['permuted'] = perf_df.hetnet.str.contains('_perm').astype(int)

In [5]:
perf_df.merge(feature_type_df).head(3)

,hetnet,feature,nonzero,auroc,auprc,feature_type,permuted
0,rephetio-v2.0,prior_prob,1.0,0.848064,0.597974,prior,0
1,rephetio-v2.0_perm-1,prior_prob,1.0,0.830700,0.603522,prior,1
2,rephetio-v2.0_perm-2,prior_prob,1.0,0.838253,0.630763,prior,1


In [6]:
def compare_permutation(df):
    unperm = df.query("permuted == 0").iloc[0, :]
    perm_df = df.query("permuted == 1")
    series = pandas.Series()
    series['nonzero'] = unperm['nonzero']
    series['auroc'] = unperm.auroc
    series['auroc_permuted'] = perm_df.auroc.mean()
    series['delta_auroc'] = series['auroc'] - series['auroc_permuted']
    ttest = scipy.stats.ttest_1samp(perm_df.auroc, unperm.auroc)
    pvalue = ttest.pvalue
    series['pval_auroc'] = pvalue
    #series['nlog10_pval_auroc'] = -math.log10(pvalue)
    return(series)

compare_df = perf_df.groupby(['feature_type', 'feature']).apply(compare_permutation).reset_index()
reject, compare_df['fdr_pval_auroc'], alphacSidak, alphacBonf = multipletests(
    pvals=compare_df.pval_auroc, method='fdr_bh')
compare_df = feature_type_df.merge(compare_df)

In [7]:
compare_df.head(3)

,feature,feature_type,nonzero,auroc,auroc_permuted,delta_auroc,pval_auroc,fdr_pval_auroc
0,prior_prob,prior,1.000000,0.848064,0.833701,0.014362,0.002286,0.011951
1,CbG,degree,0.988079,0.547779,0.549037,-0.001257,0.540220,0.646240
2,CcSE,degree,0.954702,0.564520,0.568342,-0.003822,0.044426,0.091959


In [8]:
# Save datasets
perf_df.to_csv('data/auc.tsv', sep='\t', index=False, float_format='%.5g')
compare_df.to_csv('data/auroc.tsv', sep='\t', index=False, float_format='%.5g')